In [1]:
import pandas as pd
import numpy  as np

In [55]:
DF_PREFIX = 'door'
df = pd.read_csv(f'plots/{DF_PREFIX}_stats.csv')

In [56]:
rows = ['$k$ & Noise & Base & \multicolumn{2}{c|}{$\mu$} & \multicolumn{2}{c|}{$\Sigma$} & \multicolumn{2}{c|}{$R_{XYZ}$} & \multicolumn{2}{c}{$eig$}']

def format_value(val, max_val, base_val, numformat='3.1f'):
    if val == max_val:
        return '$\mathbf{}{}{}$'.format('{', ('{:' + numformat + '}').format(val), '}')
    elif val < base_val:
        return '$\color{}{}{}{}$'.format('{red}', '{', ('{:' + numformat + '}').format(val), '}')
    return '${}$'.format(('{:' + numformat + '}').format(val))

data_rows = []

for n in sorted(df.noise.unique()):
    # acc_max = np.mean([df[df.noise == n].accuracy.max() * 100

    means = []
    eps   = []
    df_s  = df[(df.noise == n)]
    for k in sorted(df_s.k.unique()):
        df_s = df[(df.noise == n) & (df.k == k)]
        mu_mean, mu_ep = df_s[df_s.mu    > 0].accuracy.mean() * 100, df_s[df_s.mu    > 0].mean_final.mean()
        sigma_mean, sigma_ep = df_s[df_s.sigma > 0].accuracy.mean()  * 100, df_s[df_s.sigma > 0].mean_final.mean()
        rot_mean, rot_ep = df_s[(df_s.sigma > 0) & (df_s.sigma_type == "rotation")].accuracy.mean() * 100, df_s[(df_s.sigma > 0) & (df_s.sigma_type == "rotation")].mean_final.mean()
        eig_mean, eig_ep = df_s[(df_s.sigma > 0) & (df_s.sigma_type == "eigen")].accuracy.mean() * 100, df_s[(df_s.sigma > 0) & (df_s.sigma_type == "eigen")].mean_final.mean()

        means.append([mu_mean, sigma_mean, rot_mean, eig_mean])
        eps.append([mu_ep, sigma_ep, rot_ep, eig_ep])

    ep_min  = np.min(eps)
    acc_max = np.max(means)

    for k, mean, ep in zip(sorted(df.k.unique()), means, eps):
        # df_s = df[df.noise == n]
        df_s = df[(df.noise == n) & (df.k == k)]

        mu_mean, sigma_mean, rot_mean, eig_mean = mean
        mu_ep, sigma_ep, rot_ep, eig_ep = ep

        # base = df[df.noise == n].baseline.mean() * 100
        local_base = df_s.baseline.mean() * 100
        base = local_base
        lamax = max(acc_max, base)

        row  = [f'${k}$',
                f'${n:1.2f}$',
                format_value(base, lamax, base, '2.1f'),
                # f'${df_s[df_s.omega > 0].accuracy.mean() * 100:3.1f}\%$', f'${df_s[df_s.omega > 0].mean_final.mean():3.0f}$',
                format_value(mu_mean, lamax, base, '3.1f'), format_value(mu_ep, ep_min, 0, '3.0f'),
                format_value(sigma_mean, lamax, base, '3.1f'), format_value(sigma_ep, ep_min, 0, '3.0f'),
                format_value(rot_mean, lamax, base, '3.1f'), format_value(rot_ep, ep_min, 0, '3.0f'),
                format_value(eig_mean, lamax, base, '3.1f'), format_value(eig_ep, ep_min, 0, '3.0f')]
        data_rows.append((k, n, ' & '.join(row)))

table = ' \\\\ \n'.join(rows + [r for _, _, r in sorted(data_rows)])
print(table)

$k$ & Noise & Base & \multicolumn{2}{c|}{$\mu$} & \multicolumn{2}{c|}{$\Sigma$} & \multicolumn{2}{c|}{$R_{XYZ}$} & \multicolumn{2}{c}{$eig$} \\ 
$3$ & $0.00$ & $66.0$ & $73.4$ & $206$ & $77.0$ & $174$ & $75.0$ & $172$ & $79.1$ & $177$ \\ 
$3$ & $0.01$ & $45.0$ & $46.5$ & $187$ & $47.7$ & $182$ & $45.9$ & $176$ & $49.5$ & $188$ \\ 
$3$ & $0.02$ & $29.0$ & $\color{red}{22.6}$ & $158$ & $\color{red}{26.2}$ & $158$ & $\color{red}{24.4}$ & $160$ & $\color{red}{28.1}$ & $\mathbf{156}$ \\ 
$3$ & $0.03$ & $16.0$ & $\color{red}{12.9}$ & $174$ & $\color{red}{14.7}$ & $169$ & $\mathbf{15.1}$ & $169$ & $\color{red}{14.3}$ & $168$ \\ 
$5$ & $0.00$ & $48.0$ & $74.4$ & $194$ & $79.6$ & $196$ & $78.2$ & $181$ & $80.9$ & $211$ \\ 
$5$ & $0.01$ & $51.0$ & $\color{red}{49.5}$ & $208$ & $51.3$ & $224$ & $\color{red}{50.2}$ & $221$ & $52.4$ & $226$ \\ 
$5$ & $0.02$ & $28.0$ & $\color{red}{25.2}$ & $170$ & $\color{red}{26.9}$ & $189$ & $\color{red}{27.2}$ & $181$ & $\color{red}{26.6}$ & $198$ \\ 
$5$ & $0.0